<a href="https://colab.research.google.com/github/ChipDesignRashid/osoc1_core_uarch/blob/main/osoc1_core_uarch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sys

# -------------------------------------------------------------------
# CLASS 1: THE "BRAIN" (Controller)
# -------------------------------------------------------------------
class Controller:
    def decode(self, opcode, funct3):
        control_signals = {
            "reg_write_enable": False,
            "alu_op": "nop",
            "mem_read": False,
            "mem_write": False,
            "mem_to_reg": False,
            "alu_src": "imm"
        }

        if opcode == 0x13 and funct3 == 0x0: # addi
            control_signals["reg_write_enable"] = True
            control_signals["alu_op"] = "add"

        elif opcode == 0x03 and funct3 == 0x2: # lw
            control_signals["reg_write_enable"] = True
            control_signals["alu_op"] = "add"
            control_signals["mem_read"] = True
            control_signals["mem_to_reg"] = True

        return control_signals

# -------------------------------------------------------------------
# CLASS 2: THE "MUSCLE" (Datapath)
# -------------------------------------------------------------------
class RiscVDatapath:
    def __init__(self):
        self.pc = 0
        self.regs = [0] * 32
        self.instr_mem = {}
        self.data_mem = {}
        self.controller = Controller()
        self.reset()

    def reset(self):
        print("[CPU]: Resetting processor...")
        self.pc = 0
        self.regs = [0] * 32
        self.data_mem = {}
        print("[CPU]: Reset complete.")

    def load_program(self, instructions, base_address=0):
        addr = base_address
        for instr in instructions:
            self.instr_mem[addr] = instr
            addr += 4

    def dump_regs(self):
        print("--- Register Dump ---")
        for i in range(32):
            if self.regs[i] != 0:
                print(f"  x{i:<2}: 0x{self.regs[i]:08x} ({self.regs[i]})")
        print(f"  PC: 0x{self.pc:08x}")
        print("---------------------")

    # =================================================================
    # DEBUG HELPER (The Disassembler)
    # =================================================================
    def _get_instruction_debug_string(self, decoded):
        """
        Converts decoded signals back to an assembly string for display.
        This is NOT hardware; it's just for us humans.
        """
        op = decoded["opcode"]
        f3 = decoded["funct3"]
        rd = decoded["rd"]
        rs1 = decoded["rs1"]

        # Helper to show signed immediate value
        imm = decoded["imm_i"]
        if (imm >> 11) == 1: imm = imm - 4096 # Show as negative decimal

        if op == 0x13 and f3 == 0x0:
            return f"addi x{rd}, x{rs1}, {imm}"

        elif op == 0x03 and f3 == 0x2:
            return f"lw   x{rd}, {imm}(x{rs1})"

        return "unknown instruction"

    # =================================================================
    # HARDWARE BLOCKS
    # =================================================================

    def _instruction_memory_block(self, address):
        return self.instr_mem.get(address, 0)

    def _decode_block(self, instruction):
        return {
            "opcode": instruction & 0x7F,
            "rd":     (instruction >> 7) & 0x1F,
            "funct3": (instruction >> 12) & 0x7,
            "rs1":    (instruction >> 15) & 0x1F,
            "rs2":    (instruction >> 20) & 0x1F,
            "funct7": (instruction >> 25) & 0x7F,
            "imm_i":  (instruction >> 20) & 0xFFF,
            "imm_s":  ((instruction >> 25) & 0x7F) << 5 | ((instruction >> 7) & 0x1F)
        }

    def _sign_extend_block(self, imm_12bit):
        if (imm_12bit >> 11) == 1:
            return imm_12bit | 0xFFFFF000
        else:
            return imm_12bit

    def _alu_block(self, input_a, input_b, alu_op):
        if alu_op == "add":
            return (input_a + input_b) & 0xFFFFFFFF
        return 0

    def _data_memory_read_block(self, address, mem_read):
        if mem_read:
            return self.data_mem.get(address, 0)
        return 0

    # =================================================================
    # SEQUENTIAL BLOCKS
    # =================================================================

    def _data_memory_write_clock(self, address, write_data, mem_write):
        if mem_write:
            print(f"  [Memory]: Writing {write_data} to address 0x{address:x}")
            self.data_mem[address] = write_data

    def _register_file_write_clock(self, rd_addr, write_data, reg_write_enable):
        if reg_write_enable and rd_addr != 0:
            print(f"  [Writeback]: Wrote {write_data} to x{rd_addr}")
            self.regs[rd_addr] = write_data

    # =================================================================
    # TOP LEVEL WIRING
    # =================================================================

    def step(self):
        print(f"\n--- Cycle Start (PC=0x{self.pc:x}) ---")

        # 1. FETCH
        instruction_wire = self._instruction_memory_block(self.pc)
        if instruction_wire == 0: return False

        # 2. DECODE
        decoded = self._decode_block(instruction_wire)

        # --- DEBUG PRINTING ---
        # Print the assembly string right after decoding
        asm_string = self._get_instruction_debug_string(decoded)
        print(f"  [Fetch]: Instr=0x{instruction_wire:08x}  -->  {asm_string}")
        # ----------------------

        controls = self.controller.decode(decoded["opcode"], decoded["funct3"])
        # Note: Removed the verbose [Control] print to make the Assembly stand out more

        rs1_value_wire = self.regs[decoded["rs1"]]
        rs2_value_wire = self.regs[decoded["rs2"]]
        imm_32bit_wire = self._sign_extend_block(decoded["imm_i"])

        # 3. EXECUTE
        alu_input_a = rs1_value_wire
        alu_input_b = imm_32bit_wire

        alu_result_wire = self._alu_block(alu_input_a, alu_input_b, controls["alu_op"])
        # Only print if meaningful action happens
        if controls["alu_op"] != "nop":
            print(f"  [Execute]: ALU Result=0x{alu_result_wire:x}")

        # 4. MEMORY
        read_data_wire = self._data_memory_read_block(alu_result_wire, controls["mem_read"])
        if controls["mem_read"]:
            print(f"  [Memory]: Read {read_data_wire} from 0x{alu_result_wire:x}")

        # 5. WRITEBACK
        if controls["mem_to_reg"]:
            wb_data_wire = read_data_wire
        else:
            wb_data_wire = alu_result_wire

        # CLOCK TICK
        print("  [Clock Tick]: Updating state...")
        self._data_memory_write_clock(alu_result_wire, rs2_value_wire, controls["mem_write"])
        self._register_file_write_clock(decoded["rd"], wb_data_wire, controls["reg_write_enable"])

        self.pc = self.pc + 4
        return True

In [5]:
# --- "Testbench" (Top-Level Script) ---

# 1. Setup
cpu = RiscVDatapath()
cpu.reset()

# Program:
# 1. addi x5, x0, 100   (0x06400293) -> Put 100 in x5
# 2. lw x10, 8(x5)      (0x0082A503) -> Read from address (100 + 8) = 108
program_instructions = [
    0x06400293,
    0x0082A503
]
cpu.load_program(program_instructions)

# Pre-load data memory for the 'lw' test
# We put the value 777 at address 108
cpu.data_mem[108] = 777

print("\n--- Initial State ---")
cpu.dump_regs()
print(f"Data Mem[108]: {cpu.data_mem.get(108)}")

# 2. Run
print("\n--- Cycle 1 (addi) ---")
cpu.step()

print("\n--- Cycle 2 (lw) ---")
cpu.step()

# 3. Check
print("\n--- Final State ---")
cpu.dump_regs()

# Verification Logic
if cpu.regs[5] == 100 and cpu.regs[10] == 777:
    print(f"\n[SUCCESS]: x5=100 and x10=777. Both instructions worked!")
else:
    print("\n[FAILURE]: Values incorrect.")

[CPU]: Resetting processor...
[CPU]: Reset complete.
[CPU]: Resetting processor...
[CPU]: Reset complete.

--- Initial State ---
--- Register Dump ---
  PC: 0x00000000
---------------------
Data Mem[108]: 777

--- Cycle 1 (addi) ---

--- Cycle Start (PC=0x0) ---
  [Fetch]: Instr=0x06400293  -->  addi x5, x0, 100
  [Execute]: ALU Result=0x64
  [Clock Tick]: Updating state...
  [Writeback]: Wrote 100 to x5

--- Cycle 2 (lw) ---

--- Cycle Start (PC=0x4) ---
  [Fetch]: Instr=0x0082a503  -->  lw   x10, 8(x5)
  [Execute]: ALU Result=0x6c
  [Memory]: Read 777 from 0x6c
  [Clock Tick]: Updating state...
  [Writeback]: Wrote 777 to x10

--- Final State ---
--- Register Dump ---
  x5 : 0x00000064 (100)
  x10: 0x00000309 (777)
  PC: 0x00000008
---------------------

[SUCCESS]: x5=100 and x10=777. Both instructions worked!
